# Web Scraping

Técnica utilizada para extraer automáticamente datos páginas web

Pasos para el scraping
1. Especificar las urls de los sitios web y las páginas que se van a scapear.
2. Leer el archivo robots.txt de la página a visitar, para asegurarse de que está permitido.
3. Hacer una petición HTML a las URL.
4. Utilizar localizadores como expresiones regulares para extraer la información deseada del HTML.
5. Guardar los datos en un formato estructurado (como CSV o JSON).

Desventajas:
- Variabilidad
- Durabilidad



In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def arreglarPorcentaje(valor):
    for i in range(len(valor["Precio anterior"])):
        if valor["Precio anterior"][i] == '':
            valor["Porcentaje"].insert(i, "0%")
    return valor["Porcentaje"]

def obtenerTexto(elementos):
    listado = []
    for i in elementos:
      listado.append(i.getText())
    return listado


diccionario={}
pagina = requests.get("https://www.mercadolibre.com.ar/ofertas#c_id=/home/promotions-recommendations/element&c_uid=5a54f368-27bd-4ba2-9392-0dfe4d2cc8c9")
objeto_sopa = BeautifulSoup(pagina.content, 'html.parser')
nombres = objeto_sopa.find_all('p', class_='promotion-item__title')
precios = objeto_sopa.find_all('span', class_='andes-money-amount andes-money-amount--cents-superscript')
porcentaje = objeto_sopa.find_all('span', class_='promotion-item__discount-text')
precioAnterior = objeto_sopa.find_all('s', class_='andes-money-amount andes-money-amount-combo__previous-value andes-money-amount--previous andes-money-amount--cents-superscript')
diccionario["nombres"] = obtenerTexto(nombres)
diccionario["Precio"] = obtenerTexto(precios)
diccionario["Porcentaje"] = obtenerTexto(porcentaje)
diccionario["Precio anterior"] = obtenerTexto(precioAnterior)
diccionario["Porcentaje"] = arreglarPorcentaje(diccionario)
# nos quedamos con los 49 primeros
diccionario["nombres"] = diccionario["nombres"][:30]
diccionario["Precio"] = diccionario["Precio"][:30]
diccionario["Precio anterior"] = diccionario["Precio anterior"][:30]
diccionario["Porcentaje"] = diccionario["Porcentaje"][:30]

data = pd.DataFrame(diccionario)
data.to_csv("./ofertasMercadoLibre.csv")

In [3]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://www.worldometers.info/world-population/population-by-country/"
req = requests.get(url)
soup = BeautifulSoup(req.text)
data = soup.find_all("table")[0]
df_population = pd.read_html(str(data))[0]
export_csv = df_population.to_csv("tabla.csv", index = None, header = True)

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


diccionario={}
preciosActuales = []
preciosAnteriores = []
descuentos = []
nombres = []

pagina = requests.get("https://www.mercadolibre.com.ar/ofertas#c_id=/home/promotions-recommendations/element&c_uid=5a54f368-27bd-4ba2-9392-0dfe4d2cc8c9")
contenido = BeautifulSoup(pagina.content, 'html.parser')
promociones = contenido.find_all("div", class_ = "promotion-item__description", limit = 50) #obtenemos el contenido de los productos

for promocion in promociones:
  preciosActuales.append(promocion.find(class_="andes-money-amount__fraction").getText())
  if promocion.find("s"):
      preciosAnteriores.append(promocion.find("s").find_next("span", class_="andes-money-amount__fraction").getText())
  else:
      preciosAnteriores.append(np.nan)
  # preciosAnteriores.append(promocion.s.find_next("span", class_="andes-money-amount__fraction").getText())
  descuentos.append(promocion.find(class_="promotion-item__discount-text").getText())
  nombres.append(promocion.find(class_="promotion-item__title").getText())

diccionario["nombre"] = nombres
diccionario["descuento"] = descuentos
diccionario["precio actual"] = preciosActuales
diccionario["precio anterior"] = preciosAnteriores
data = pd.DataFrame(diccionario)
data.to_csv("./ofertasMercadoLibreb.csv")
